In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.float_format='{:.2f}'.format
plt.style.use('seaborn')
import sklearn as skl
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
#for the tuner
import tensorflow as tf
import keras_tuner as kt
#
import io
import shutil
import os
import warnings
warnings.filterwarnings("ignore")

C:\Users\rober\AppData\Local\Temp\ipykernel_280\2887631212.py:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [2]:
fpth='mushroom_data_clean.csv'
df=pd.read_csv(fpth)
df.head()
# df.describe()
# df.info()
# for c in df.columns:
#     print(c,df[c].unique())

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above,stalk-color-below,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,e


In [3]:
df.drop(columns=['stalk-root'],inplace=True)
df.head()

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above,stalk-color-below,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,e


In [4]:
df['poisonous']=df['class']
df['poisonous']=df['poisonous'].apply(lambda x:1 if x in 'p' else 0)
y=df['poisonous']
df.drop(columns=['class','poisonous'],inplace=True)
X=df

In [5]:
for c in df.columns:
    print(c,df[c].unique())

cap-shape ['x' 'b' 's' 'f' 'k' 'c']
cap-surface ['s' 'y' 'f' 'g']
cap-color ['n' 'y' 'w' 'g' 'e' 'p' 'b' 'u' 'c' 'r']
bruises ['t' 'f']
odor ['p' 'a' 'l' 'n' 'f' 'c' 'y' 's' 'm']
gill-attachment ['f' 'a']
gill-spacing ['c' 'w']
gill-size ['n' 'b']
gill-color ['k' 'n' 'g' 'p' 'w' 'h' 'u' 'e' 'b' 'r' 'y' 'o']
stalk-shape ['e' 't']
stalk-surface-above ['s' 'f' 'k' 'y']
stalk-surface-below ['s' 'f' 'y' 'k']
stalk-color-above ['w' 'g' 'p' 'n' 'b' 'e' 'o' 'c' 'y']
stalk-color-below ['w' 'p' 'g' 'b' 'n' 'e' 'y' 'o' 'c']
veil-type ['p']
veil-color ['w' 'n' 'o' 'y']
ring-number ['o' 't' 'n']
ring-type ['p' 'e' 'l' 'f' 'n']
spore-print-color ['k' 'n' 'u' 'h' 'w' 'r' 'o' 'y' 'b']
population ['s' 'n' 'a' 'v' 'y' 'c']
habitat ['u' 'g' 'm' 'd' 'p' 'w' 'l']


In [6]:
#reduction
NUMBER_OF_COLUMNS_TO_KEEP=25
DELETE_CUTOFF_BASED_ON_FEATURES=8
LE_THAN=False
    #93% with 5 columns

#tuner parameters
ITERATIONS=5
EPOCHS=15
MAX_LAYERS=10
MAX_NODES=20

In [7]:
for c in df.columns:
    number_of_unique=len(df[c].unique())
    print(c,number_of_unique)
    if LE_THAN and number_of_unique<=DELETE_CUTOFF_BASED_ON_FEATURES:
        df.drop(c,axis=1,inplace=True)
    elif not LE_THAN and number_of_unique>=DELETE_CUTOFF_BASED_ON_FEATURES:
        df.drop(c,axis=1,inplace=True)

cap-shape 6
cap-surface 4
cap-color 10
bruises 2
odor 9
gill-attachment 2
gill-spacing 2
gill-size 2
gill-color 12
stalk-shape 2
stalk-surface-above 4
stalk-surface-below 4
stalk-color-above 9
stalk-color-below 9
veil-type 1
veil-color 4
ring-number 3
ring-type 5
spore-print-color 9
population 6
habitat 7


In [8]:
X=pd.get_dummies(X)
X

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,0,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
8120,0,0,0,0,0,1,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
8121,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
8122,0,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,1,0,0,0,0


In [9]:
cor_matrix=X.corr().abs() 
upper_tri=cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool)) 
to_drop=[column for column in upper_tri.columns if any(upper_tri[column] > 0.95)] 
X.drop(to_drop,axis=1,inplace=True)
X.head()

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,0,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [10]:
def drop_more_columns(X,y):
    k=NUMBER_OF_COLUMNS_TO_KEEP
    X_new = SelectKBest(f_regression, k=k).fit_transform(X, y)
    selected_features = X.columns[SelectKBest(f_regression, k=k).fit(X, y).get_support()]
    delete_these = set(X.columns)-set(selected_features)
    delete_these=list(delete_these)
    for col in delete_these:
        print('deleting %s'%col)
        X.drop(col,axis=1,inplace=True)
drop_more_columns(X,y)
X.head()

deleting cap-surface_s
deleting cap-surface_g
deleting ring-type_f
deleting stalk-surface-above_y
deleting cap-shape_s
deleting cap-shape_f
deleting cap-shape_c
deleting veil-color_y
deleting cap-surface_y
deleting gill-attachment_a
deleting population_y
deleting veil-type_p
deleting habitat_u
deleting stalk-surface-above_f
deleting habitat_d
deleting veil-color_n
deleting cap-shape_x
deleting stalk-surface-below_y
deleting veil-color_o
deleting ring-number_n
deleting stalk-shape_e
deleting stalk-surface-below_f


,cap-shape_b,cap-shape_k,cap-surface_f,bruises_f,gill-spacing_c,gill-size_b,stalk-surface-above_k,stalk-surface-above_s,stalk-surface-below_k,stalk-surface-below_s,...,population_a,population_c,population_n,population_s,population_v,habitat_g,habitat_l,habitat_m,habitat_p,habitat_w
0,0,0,0,0,1,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,1,1,0,1,0,1,...,0,0,1,0,0,1,0,0,0,0
2,1,0,0,0,1,1,0,1,0,1,...,0,0,1,0,0,0,0,1,0,0
3,0,0,0,0,1,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,1,0,1,0,1,0,1,...,1,0,0,0,0,1,0,0,0,0


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1,stratify=y)

In [12]:
try:
    dirpath=('./untitled_project')
    shutil.rmtree(dirpath)
    os.mkdir(dirpath)
except: pass

In [13]:
# Create a method that creates a new Sequential model with hyperparameter options
INPUT_DIM=X_train.shape[1]
def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    activation = hp.Choice('activation',['relu','tanh','sigmoid','softmax'])
    hp_optimizer = hp.Choice('optimizer', values=['sgd', 'rmsprop', 'adam'])
    hp_lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    if hp_optimizer == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=hp_lr)
    elif hp_optimizer == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=hp_lr)
    elif hp_optimizer == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=hp_lr)
    else:
        raise
    
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=MAX_NODES,
        step=2), activation=activation, input_dim=INPUT_DIM))
    for i in range(hp.Int('num_layers', 1, MAX_LAYERS)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=MAX_NODES,
            step=2),
            activation=activation))
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    nn_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return nn_model

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=EPOCHS,
    hyperband_iterations=ITERATIONS)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(6093, 25) (6093,) (2031, 25) (2031,)


In [ ]:
tuner.search(X_train,y_train,epochs=EPOCHS,validation_data=(X_test,y_test))

Trial 93 Complete [00h 00m 05s]
val_accuracy: 0.768094539642334

Best val_accuracy So Far: 0.9867060780525208
Total elapsed time: 00h 09m 21s

Search: Running Trial #94

Value             |Best Value So Far |Hyperparameter
sigmoid           |relu              |activation
adam              |adam              |optimizer
0.01              |0.001             |learning_rate
5                 |19                |first_units
9                 |6                 |num_layers
1                 |17                |units_0
7                 |13                |units_1
7                 |15                |units_2
9                 |1                 |units_3
5                 |15                |units_4
17                |11                |units_5
1                 |17                |units_6
3                 |15                |units_7
5                 |13                |units_8
5                 |3                 |units_9
2                 |15                |tuner/epochs
0                 

In [ ]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
best_hyper.values